In [126]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import FeatureHasher
import category_encoders as ce
import pandas as pd
import numpy as np

In [137]:
#Cargo los sets
X_train = pd.read_csv('data/X_train.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'object',\
                                           'ciudad':'object', 'provincia':'object',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'object',\
                                           'ciudad':'object', 'provincia':'object',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [128]:
randomForest = RandomForestRegressor(n_estimators=100, n_jobs= 5, oob_score=True, min_samples_leaf=2, max_features= 0.5)

In [138]:
X_train

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,...,ciudad__Tahmek,ciudad__Tangancícuaro,ciudad__Teteles de Avila Castillo,ciudad__Timucuy,ciudad__Tinum,ciudad__Tlaxco,ciudad__Uayma,ciudad__Villa Purificación,ciudad__Villanueva,ciudad__Xalatlaco
id,,,,,,,,,,,,,,,,,,,,,
120542,8.121536,3.000000,2.000000,3.000000,130.000000,144.000000,346913.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
11955,0.000000,3.000000,1.000000,3.000000,170.000000,130.000000,106912.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
38491,5.000000,2.000000,1.000000,1.000000,64.000000,64.000000,23835.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
235712,8.121536,3.000000,0.000000,1.000000,50.000000,176.812747,55785.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
273251,4.000000,2.000000,1.000000,1.000000,50.000000,50.000000,24517.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27738,10.000000,3.000000,2.000000,2.000000,150.000000,176.812747,24613.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
250425,0.000000,2.903351,1.547407,2.133895,174.122525,246.000000,84345.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0
49430,14.000000,2.000000,1.000000,1.000000,57.000000,57.000000,59712.0,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [102]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [103]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [104]:
def print_score(r,train_set,test_set,train_target,test_target,F):
    res = [F(r.predict(train_set), train_target),
           F(r.predict(test_set), test_target),
           r.score(train_set, train_target), r.score(test_set, test_target)]
    if hasattr(r, 'oob_score_'): res.append(r.oob_score_)
    print(res)

In [105]:
print("Cantidad de columnas train:",X_train.columns.size)

Cantidad de columnas train: 18


In [106]:
print("Cantidad de columnas test:",X_test.columns.size)

Cantidad de columnas test: 18


In [107]:
cat = ['tipodepropiedad','ciudad','provincia']

In [108]:
suma = 0

for categoria in cat:
    suma += X_train[categoria].nunique()
    
print('Cantidad de valores unicos de categorias:',suma)

Cantidad de valores unicos de categorias: 869


In [109]:
no_categoricas = X_train.columns.size
print(suma + no_categoricas-3)

884


In [110]:
X_train.shape

(179691, 18)

In [111]:
X_train_cat = X_train.iloc[:,:3]
X_train_cat['precio_log'] = y_train_log
X_train_cat

,tipodepropiedad,ciudad,provincia,precio_log
id,,,,
120542,Casa,Mazatlán,Sinaloa,14.483084
11955,Casa,Xalapa,Veracruz,14.430697
38491,Apartamento,Cuauhtémoc,Distrito Federal,14.200774
235712,Local Comercial,Atizapán de Zaragoza,Edo. de México,14.119313
273251,Apartamento,La Magdalena Contreras,Distrito Federal,13.997833
...,...,...,...,...
27738,Apartamento,Miguel Hidalgo,Distrito Federal,14.940440
250425,Terreno,Querétaro,Querétaro,13.871703
49430,Apartamento,Nezahualcóyotl,Edo. de México,12.906694


In [112]:
X_test_cat = X_test.iloc[:,:3]
X_test_cat['precio_log'] = y_test_log
X_test_cat

,tipodepropiedad,ciudad,provincia,precio_log
id,,,,
13184,Apartamento,Villahermosa,Tabasco,13.698978
238860,Apartamento,Cuauhtémoc,Distrito Federal,13.997833
166613,Oficina comercial,San Luis Potosí,San luis Potosí,14.077876
132784,Casa,Tampico,Tamaulipas,14.648420
29982,Casa,Tlalpan,Distrito Federal,15.598902
...,...,...,...,...
126014,Terreno,Lerma,Edo. de México,15.150512
17229,Apartamento,Tlalpan,Distrito Federal,14.316286
180966,Apartamento,Guadalajara,Jalisco,14.220976


In [114]:
ce_bin = ce.BinaryEncoder(cols = ['tipodepropiedad','provincia','ciudad'])
X_train_cat = ce_bin.fit_transform(X_train_cat,X_train_cat['precio_log'])
X_train_cat.drop(columns=['precio_log'],inplace=True)

In [115]:
X_train_cat.shape

(179691, 23)

In [116]:
ce_bin = ce.BinaryEncoder(cols = ['tipodepropiedad','provincia','ciudad'])
X_test_cat = ce_bin.fit_transform(X_test_cat,X_test_cat['precio_log'])
X_test_cat.drop(columns=['precio_log'],inplace=True)

In [117]:
X_test_cat.shape

(59897, 23)

In [118]:
X_train.drop(columns=['tipodepropiedad','provincia','ciudad'],inplace=True)

In [119]:
X_test.drop(columns=['tipodepropiedad','provincia','ciudad'],inplace=True)

In [120]:
merge_train = pd.merge(X_train_cat,X_train,left_index=True, right_index=True)
merge_train.head()

,tipodepropiedad_0,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,ciudad_0,ciudad_1,ciudad_2,ciudad_3,...,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,año,mes,dia
id,,,,,,,,,,,,,,,,,,,,,
120542,0,0,0,0,0,1,0,0,0,0,...,144.000000,346913.0,False,False,False,False,False,2016,12,15
11955,0,0,0,0,0,1,0,0,0,0,...,130.000000,106912.0,False,False,False,False,False,2013,3,19
38491,0,0,0,0,1,0,0,0,0,0,...,64.000000,23835.0,False,False,False,False,False,2014,10,6
235712,0,0,0,0,1,1,0,0,0,0,...,176.812747,55785.0,False,False,False,False,False,2016,5,17
273251,0,0,0,0,1,0,0,0,0,0,...,50.000000,24517.0,False,False,False,True,False,2016,12,28


In [121]:
merge_test = pd.merge(X_test_cat,X_test,left_index=True, right_index=True)
merge_test.head()

,tipodepropiedad_0,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,ciudad_0,ciudad_1,ciudad_2,ciudad_3,...,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,año,mes,dia
id,,,,,,,,,,,,,,,,,,,,,
13184,0,0,0,0,0,1,0,0,0,0,...,90.0,130399.0,False,False,False,True,True,2014,4,12
238860,0,0,0,0,0,1,0,0,0,0,...,60.0,23832.0,False,False,False,True,True,2014,3,5
166613,0,0,0,0,1,0,0,0,0,0,...,54.0,87712.0,False,False,False,True,True,2015,11,16
132784,0,0,0,0,1,1,0,0,0,0,...,312.0,104601.0,False,False,False,False,False,2013,3,8
29982,0,0,0,0,1,1,0,0,0,0,...,200.0,24810.0,False,False,False,False,False,2016,12,1


In [122]:
merge_train.to_csv('data/X_train_bin.csv')
merge_test.to_csv('data/X_test_bin.csv')

In [124]:
%time randomForest.fit(merge_train, y_train_log)

CPU times: user 1min 52s, sys: 442 ms, total: 1min 52s
Wall time: 25.4 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
                      oob_score=True, random_state=None, verbose=0,
                      warm_start=False)

In [125]:
%time print_score(randomForest,merge_train,merge_test,y_train_log,y_test_log,RMSE)

[0.1687285977694726, 0.5110632627979387, 0.9593424154512813, 0.6300245536923211, 0.8407385659357688]
CPU times: user 30.6 s, sys: 192 ms, total: 30.8 s
Wall time: 7.19 s


In [123]:
######################################################################

In [14]:
X_train.head()

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,año,mes,dia
id,,,,,,,,,,,,,,,,,,
120542,Casa,Mazatlán,Sinaloa,8.121536,3.0,2.0,3.0,130.0,144.000000,346913.0,False,False,False,False,False,2016,12,15
11955,Casa,Xalapa,Veracruz,0.000000,3.0,1.0,3.0,170.0,130.000000,106912.0,False,False,False,False,False,2013,3,19
38491,Apartamento,Cuauhtémoc,Distrito Federal,5.000000,2.0,1.0,1.0,64.0,64.000000,23835.0,False,False,False,False,False,2014,10,6
235712,Local Comercial,Atizapán de Zaragoza,Edo. de México,8.121536,3.0,0.0,1.0,50.0,176.812747,55785.0,False,False,False,False,False,2016,5,17
273251,Apartamento,La Magdalena Contreras,Distrito Federal,4.000000,2.0,1.0,1.0,50.0,50.000000,24517.0,False,False,False,True,False,2016,12,28


In [ ]:
X_train